In [ ]:
import nibiru
import nibiru.msg # Messages package for transactions
import json

In [ ]:
nibiru.__version__ # 0.16.0 is compatible with nibiru chain v0.16

In [ ]:
network = nibiru.Network.customnet()

# Transaction types

There are 3 ways to post transactions with the python Sdk, having to do with the synchronization needed by the user.

```python
class TxType(Enum):
    """
    The TxType allows you to chose what type of synchronization you want to use to send transaction
    """

    SYNC = 1
    """
    The CLI waits for a CheckTx execution response only.
    Each full-node that receives a transaction sends a CheckTx to the application layer to check for validity, and
    receives an abci.ResponseCheckTx. If the Tx passes the checks, it is held in the nodes' Mempool , an in-memory pool
    of transactions unique to each node pending inclusion in a block - honest nodes will discard Tx if it is found to
    be invalid.

    Prior to consensus, nodes continuously check incoming transactions and gossip them to their peers.
    """

    ASYNC = 2
    """
    The CLI returns immediately (transaction might fail silently).
    If you send a transaction with this option, it is recommended to query the transaction output using the hash of the
    transaction given by the output of the tx call.
    """

    BLOCK = 3
    """
    The tx function will wait unitl the tx is be committed in a block.
    This have the effect of having the full log of the transaction available with the output of the method. These logs
    will include information as to coin sent and received, states changed etc.
    """
```

We can adjust our agent to open positions with any of those options

In [ ]:
agent_mnemonic = "vapor maximum leg hybrid purchase cash arrow credit pink produce already stool umbrella evil exhaust doll tell category cheese history gossip clown favorite"

In [ ]:
tx_config = nibiru.TxConfig(tx_type=nibiru.TxType.BLOCK)
trader = nibiru.Sdk.authorize(agent_mnemonic).with_network(network).with_config(tx_config)

Our trader don't have any fund, so if we try to open a position it will fail saying that the address does not exist (never registered with the auth module of the chain)

In [ ]:
pair = "ubtc:unusd"

In [ ]:
trader.tx.execute_msgs(
    nibiru.msg.perp.MsgOpenPosition( # we will explain better this below
        sender=trader.address,
        token_pair=pair,
        side=nibiru.Side.BUY,
        quote_asset_amount=10,
        leverage=10,
        base_asset_amount_limit=0,
    )
)

For our example, let's fund our trader with some fund from the validator node of our local tesnet.

In [ ]:
VALIDATOR_MNEMONIC="guard cream sadness conduct invite crumble clock pudding hole grit liar hotel maid produce squeeze return argue turtle know drive eight casino maze host"

In [ ]:
validator = (
    nibiru.Sdk
        .authorize(VALIDATOR_MNEMONIC) # This allows us to recover the wallet with a mnemonic
        .with_network(network)
        .with_config(tx_config)
)

Now that we have a validator agent, let's send some fund with a MsgSend transaction

In [ ]:
tx_output = validator.tx.execute_msgs(
    nibiru.msg.bank.MsgSend(
        from_address=validator.address,
        to_address=trader.address,
        coins=[nibiru.Coin(10000, "unibi"), nibiru.Coin(10000, "unusd")]
    )
)

We can now query the balance of the trader address using the query we saw in the previous notebook

In [ ]:
trader.query.get_bank_balances(trader.address)

Let's try to open again a position

In [ ]:
# New bug found today require us to run this command again.
trader = nibiru.Sdk.authorize(agent_mnemonic).with_network(network).with_config(tx_config)


tx_output = trader.tx.execute_msgs(
    nibiru.msg.perp.MsgOpenPosition( # we will explain better this below
        sender=trader.address,
        token_pair=pair,
        side=nibiru.Side.BUY,
        quote_asset_amount=10,
        leverage=10,
        base_asset_amount_limit=0,
    )
)

We can take a look at the log of the transaction by reading the txoutput

In [ ]:
print(json.dumps(tx_output["rawLog"], indent=4))

Documentation for msgs can be found there: https://nibiru-py.readthedocs.io/en/latest/nibiru.msg.html

In [ ]:
trader.query.perp.position(token_pair=pair, trader=trader.address)